<a href="https://colab.research.google.com/github/cesarrivera52/Actividad-4/blob/main/Actividad4_RegresionLogistica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#César Alejandro Rivera Guzmán  A01567012
#Actividad 4
#Regresión Logística 3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from google.colab import files
files.upload()

In [5]:
data=pd.read_excel('microretailer_mit_lift_lab_actualizado (1).xlsx')

In [6]:
#Eliminar columnas innecesarias
datam1=data.drop(["_record_id","_title","_server_updated_at","234_willingness_participate_workshop","_geometry","_latitude","_longitude","229_store_picture",],axis=1)

In [7]:
datam1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 98 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   _updated_by                                                 330 non-null    object 
 1   228_store_name                                              329 non-null    object 
 2   232_type_of_store                                           330 non-null    object 
 3   108_does_the_micro_retailer_has_a_barred_window_            330 non-null    object 
 4   99_does_the_micro_retailer_exhibits_products_outside_       330 non-null    object 
 5   102_does_the_micro_retailer_sells_fresh_products_           330 non-null    object 
 6   97_number_of_customers_in_store                             330 non-null    int64  
 7   268_number_fridges                                          321 non-null    float64
 8   

In [8]:
#Lo que hacemos aqui es sustituir los valores núlos por la palabra Anónimo
datam1["319_shopkeepers_last_name"]=datam1["319_shopkeepers_last_name"].fillna("Anónimo")

In [9]:
#Lo que hacemos aqui es sustituir los valores núlos por la palabra Anónimo
datam1["318_shopkeeper_name"]=datam1["318_shopkeeper_name"].fillna("Anónimo")

In [10]:
#Lo que hacemos aqui es sustituir los valores núlos por la palabra Anónimo
datam1["235_email"]=datam1["235_email"].fillna("Anónimo")

In [11]:
#Lo que hacemos aqui es sustituir los valores núlos por la palabra Anónimo
datam1["236_whatsapp"]=datam1["236_whatsapp"].fillna("Anónimo")

In [12]:
#Lo que hacemos aqui es sustituir los valores núlos por el número 0
datam1["271_card_fee"]=datam1["271_card_fee"].fillna("0")

In [13]:
#Lo que hacemos aqui es sustituir los valores núlos por el número 0
datam1["273_platform_extra_charge"]=datam1["273_platform_extra_charge"].fillna("0")

In [14]:
#Lo que hacemos aqui es sustituir los valores núlos por el número 0
datam1["274_platform_fee"]=datam1["274_platform_fee"].fillna("0")

In [15]:
#Lo que hacemos aqui es sustituir los valores núlos por el número 0
datam1["275_platform_days_receive_money"]=datam1["275_platform_days_receive_money"].fillna("0")

In [16]:
#Filtro para separar variables cuantitativas
fdatam_cual=datam1.iloc[:,[0,1,2,3,4,5,8,10,11,12,13,16,17,18,19,20,21,23,25,27,28,29,30,31,32,33,39,40,41,43,45,46,47,48,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,81,82,84,85,86,87,88,89,90,91,92,93,94,95,96,97]]

In [17]:
#Sustituir valores nulos por valores no nulos hacia adelante forward fill" ("ffill")
fdatam_cual=fdatam_cual.fillna(method="bfill")

In [18]:
#Sustituir valores nulos por valores no nulos hacia adelante forward fill" ("ffill")
fdatam_cual=fdatam_cual.fillna(method="ffill")

In [19]:
#Identificar valores nulos por columna
valores_nulos=fdatam_cual.isnull().sum()
valores_nulos

_updated_by                                              0
228_store_name                                           0
232_type_of_store                                        0
108_does_the_micro_retailer_has_a_barred_window_         0
99_does_the_micro_retailer_exhibits_products_outside_    0
                                                        ..
267_technology_investment_customer_preferences           0
318_shopkeeper_name                                      0
319_shopkeepers_last_name                                0
236_whatsapp                                             0
235_email                                                0
Length: 76, dtype: int64

In [20]:
#Filtro para separar variables cuantitativas
fdatam_cuan=datam1.iloc[:,[6,7,9,14,15,22,24,26,34,35,36,37,38,42,44,49,50,51,52,53,71,83]]

In [21]:
#Método aplicando Cuartiles. Encuentro cuartiles 0.25 y 0.75
y=fdatam_cuan

percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior_iqr= percentile75 + 1.5*iqr
Limite_Inferior_iqr= percentile25 - 1.5*iqr
print("Limite superior permitido", Limite_Superior_iqr)
print("Limite inferior permitido", Limite_Inferior_iqr)

Limite superior permitido 97_number_of_customers_in_store                       5.000
268_number_fridges                                    5.000
104_how_many_shelves_does_the_micro_retailer_have    13.500
2_current_permanent_employees                         3.500
4_number_permanent_employees_last_year                5.000
103_number_own_fridges                                5.375
145_number_direct_competitors                        10.625
24_burnout                                           12.000
272_card_days_receive_money                           1.000
276_expected_days_informal_credit                    34.500
36_credit_default_rate                                5.000
315_frequency_topups                                 17.625
316_frequency_utility_bills                          16.000
317_home_deliveries                                  25.750
62_frequency_only_services                           17.250
78_number_home_deliveries_week                        6.000
172_supplier_f

In [22]:
data3=fdatam_cuan[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
data3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  """Entry point for launching an IPython kernel.


,97_number_of_customers_in_store,268_number_fridges,104_how_many_shelves_does_the_micro_retailer_have,2_current_permanent_employees,4_number_permanent_employees_last_year,103_number_own_fridges,145_number_direct_competitors,24_burnout,271_card_fee,272_card_days_receive_money,...,275_platform_days_receive_money,276_expected_days_informal_credit,36_credit_default_rate,315_frequency_topups,316_frequency_utility_bills,317_home_deliveries,62_frequency_only_services,78_number_home_deliveries_week,172_supplier_frequency,163_number_high_perishable_products
0,1.0,3.0,3.0,2.0,NaN,3.0,5.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,2.0,10.0
1,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.0,0.0,0.0,0.0,NaN,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
4,3.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0.0,1.0,7.0,NaN,NaN,NaN,5.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,5.0,0.0
326,1.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
327,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#Segundo método de sustitución de valores nulos
#Sustituir valores nulos con mediana
data4=data3.fillna(round(fdatam_cuan.median(),1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
#Segundo método de sustitución de valores nulos
#Sustituir valores nulos con mediana
data5=data4.fillna(round(fdatam_cuan.mean(),1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
#Tercer método de sustitución de valores nulos
#Sustituir valores nulos por un valor numérico en  concreto 
data5["271_card_fee"]= data5["271_card_fee"].fillna(0) 

In [26]:
#Tercer método de sustitución de valores nulos
#Sustituir valores nulos por un valor numérico en  concreto 
data5["273_platform_extra_charge"]= data5["273_platform_extra_charge"].fillna(0) 

In [27]:
#Tercer método de sustitución de valores nulos
#Sustituir valores nulos por un valor numérico en  concreto 
data5["274_platform_fee"]= data5["274_platform_fee"].fillna(0) 

In [28]:
#Identificar valores nulos por columna
valores_nulos=data5.isnull().sum()
valores_nulos

97_number_of_customers_in_store                      0
268_number_fridges                                   0
104_how_many_shelves_does_the_micro_retailer_have    0
2_current_permanent_employees                        0
4_number_permanent_employees_last_year               0
103_number_own_fridges                               0
145_number_direct_competitors                        0
24_burnout                                           0
271_card_fee                                         0
272_card_days_receive_money                          0
273_platform_extra_charge                            0
274_platform_fee                                     0
275_platform_days_receive_money                      0
276_expected_days_informal_credit                    0
36_credit_default_rate                               0
315_frequency_topups                                 0
316_frequency_utility_bills                          0
317_home_deliveries                                  0
62_frequen

In [29]:
data5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 22 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   97_number_of_customers_in_store                    330 non-null    float64
 1   268_number_fridges                                 330 non-null    float64
 2   104_how_many_shelves_does_the_micro_retailer_have  330 non-null    float64
 3   2_current_permanent_employees                      330 non-null    float64
 4   4_number_permanent_employees_last_year             330 non-null    float64
 5   103_number_own_fridges                             330 non-null    float64
 6   145_number_direct_competitors                      330 non-null    float64
 7   24_burnout                                         330 non-null    float64
 8   271_card_fee                                       330 non-null    int64  
 9   272_card_d

In [30]:
#Cargar archivo csv desde seaborn
df=(data5)
df.head(5)

,97_number_of_customers_in_store,268_number_fridges,104_how_many_shelves_does_the_micro_retailer_have,2_current_permanent_employees,4_number_permanent_employees_last_year,103_number_own_fridges,145_number_direct_competitors,24_burnout,271_card_fee,272_card_days_receive_money,...,275_platform_days_receive_money,276_expected_days_informal_credit,36_credit_default_rate,315_frequency_topups,316_frequency_utility_bills,317_home_deliveries,62_frequency_only_services,78_number_home_deliveries_week,172_supplier_frequency,163_number_high_perishable_products
0,1.0,3.0,3.0,2.0,2.0,3.0,5.0,2.0,0,1.0,...,0.0,8.0,1.0,6.5,2.0,5.0,10.0,2.0,2.0,10.0
1,2.0,0.0,0.0,2.0,2.0,1.0,3.0,4.0,0,1.0,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0
2,2.0,0.0,0.0,2.0,2.0,1.0,3.0,4.0,0,1.0,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0
3,1.0,0.0,0.0,0.0,0.0,1.0,10.0,4.0,0,1.0,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,1.0,0.0
4,3.0,2.0,3.0,2.0,2.0,1.0,3.0,4.0,0,1.0,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0


In [31]:
#Con el comando pd.concat juntamos las 2 variables que tenemos
Datos_limpios = pd.concat([fdatam_cual, data5], axis=1)
Datos_limpios

,_updated_by,228_store_name,232_type_of_store,108_does_the_micro_retailer_has_a_barred_window_,99_does_the_micro_retailer_exhibits_products_outside_,102_does_the_micro_retailer_sells_fresh_products_,184_store_devices,pictures_of_shelves_if_possible,hola_somos_estudiantes_del_tec_de_monterrey_estamos_realiz,le_comento_que_toda_esta_informacin_es_confidencial,...,275_platform_days_receive_money,276_expected_days_informal_credit,36_credit_default_rate,315_frequency_topups,316_frequency_utility_bills,317_home_deliveries,62_frequency_only_services,78_number_home_deliveries_week,172_supplier_frequency,163_number_high_perishable_products
0,Uriel Bonilla,Mini súper 7 hermanos,Grocery store (aka. nanostore),no,yes,yes,"Security cameras,Smartphone",6 Photos,yes,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,10.0,2.0,2.0,10.0
1,Uriel Bonilla,Super papas Lulu,Grocery store (aka. nanostore),no,yes,yes,Smartphone,6 Photos,no,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0
2,Uriel Bonilla,Arte Paquistán,Grocery store (aka. nanostore),no,yes,no,"Analog cash register,Smartphone",6 Photos,no,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0
3,Uriel Bonilla,El vestir,Grocery store (aka. nanostore),no,yes,no,Smartphone,6 Photos,yes,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,1.0,0.0
4,Fernando Báez,Antojitos Tony,Micro-restaurant (aka. fondita),no,no,yes,"Security cameras,Smartphone",6 Photos,no,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,Francisco Adán,Xuti Galería,productos tradicionales,no,yes,no,Analog cash register,1 Photo,yes,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,1.0,1.0,5.0,0.0
326,Emilio Garza,July Store,Grocery store (aka. nanostore),yes,no,no,"Analog cash register,Smartphone",1 Photo,no,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0
327,Emilio Garza,La fonda,Micro-restaurant (aka. fondita),yes,no,yes,Smartphone,1 Photo,no,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0
328,Francisco Adán,"Birria de Res ""Hermanos Negreros 2"" estilo Tij...",Micro-restaurant (aka. fondita),no,no,yes,Analog cash register,1 Photo,no,yes,...,0.0,8.0,1.0,6.5,2.0,5.0,3.0,2.0,2.0,2.0


In [32]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= Datos_limpios[['78_number_home_deliveries_week', '172_supplier_frequency', '163_number_high_perishable_products']]
Var_Dep= Datos_limpios['102_does_the_micro_retailer_sells_fresh_products_']

In [33]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [34]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.96, random_state =None)

In [35]:
#Se escalan todos los datos
escalar = StandardScaler()

In [36]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [37]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [38]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [39]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 

In [40]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[181   1]
 [131   4]]


In [41]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="yes")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8


In [42]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.583596214511041


In [43]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.02962962962962963


In [44]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="yes")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.05714285714285714
